In [1]:
# Usaremos las bibliotecas sqlite3 y pandas para trabajar con la base de datos
import pandas as pd
import sqlite3

# Establemos la conexión con la base de datos y configuramos pandas para que muestre todas las columnas
conn = sqlite3.connect('./Data/sql-murder-mystery.db')
pd.set_option('display.max_colwidth', None)

In [2]:
query = '''
SELECT description 
FROM crime_scene_report
WHERE date = '20180115' AND type = 'murder' AND city = 'SQL City';
'''
pd.read_sql_query(query, conn)

,description
0,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


In [3]:
query = '''
SELECT id, name
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'
'''
pd.read_sql_query(query, conn)

,id,name
0,16371,Annabel Miller


In [4]:
query = '''
SELECT id, name
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC 
LIMIT 1
'''
pd.read_sql_query(query, conn)

,id,name
0,14887,Morty Schapiro


In [5]:
query = '''
SELECT transcript
FROM interview
WHERE person_id = '16371'
'''
pd.read_sql_query(query, conn)

,transcript
0,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [6]:
query = '''
SELECT transcript
FROM interview
WHERE person_id = '14887'
'''
pd.read_sql_query(query, conn)

,transcript
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [7]:
query = '''
SELECT membership_id
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%' AND check_in_date = '20180109'
'''
pd.read_sql_query(query, conn)

,membership_id
0,48Z7A
1,48Z55


In [8]:
query = '''
SELECT *
FROM get_fit_now_member
WHERE id = '48Z7A' OR id='48Z55'
'''
pd.read_sql_query(query, conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


In [10]:
query = '''
SELECT p.name, d.plate_number
FROM person AS p
JOIN drivers_license AS d ON p.license_id = d.id
WHERE p.name = 'Joe Germuska' OR p.name = 'Jeremy Bowers'
'''
pd.read_sql_query(query, conn)

,name,plate_number
0,Jeremy Bowers,0H42W2


In [ ]:
query = '''
SELECT transcript
FROM interview
WHERE person_id = '67318'
'''
pd.read_sql_query(query, conn)

,transcript
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [13]:
query = '''
SELECT p.name, d.height, d.hair_color, d.car_make, d.car_model, d.gender
FROM drivers_license AS d
JOIN person AS p ON d.id = p.license_id
WHERE d.height BETWEEN 65 AND 67
AND d.hair_color = 'red'
AND d.gender = 'female'
AND d.car_make = 'Tesla'
AND d.car_model = 'Model S'
AND p.id IN (
	SELECT f.person_id
	FROM facebook_event_checkin AS f
	WHERE f.event_name = 'SQL Symphony Concert'
);
'''
pd.read_sql_query(query, conn)

,name,height,hair_color,car_make,car_model,gender
0,Miranda Priestly,66,red,Tesla,Model S,female
